In [1]:
%load_ext autoreload
%autoreload 2


import mahotas as mh
import numpy as np
import os
import time

from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
%matplotlib inline

import mlproof as mlp

In [2]:
PATCH_PATH = '/home/d/patches/cylinder_small1_rgba/'

In [7]:
def generate_patches(start_slice, end_slice):

    patch_index = 0

    all_patches_count = 0
    all_error_patches = []
    all_correct_patches = []
    
    for z in range(start_slice, end_slice):

        t0 = time.time()
        print 'working on slice', z
        input_image, input_prob, input_mask, input_gold, input_rhoana = mlp.Util.read_section('/home/d/data/cylinder/',z)

        error_patches, patches = mlp.Patch.patchify_maxoverlap(input_image, input_prob, input_mask, input_rhoana, input_gold, sample_rate=1)

        print 'Generated', len(error_patches), 'split error patches and', len(patches), ' correct patches in', time.time()-t0, 'seconds.'
        
        
        all_error_patches += error_patches
        all_correct_patches += patches
        
    
    
    NO_PATCHES = len(all_error_patches) + len(all_correct_patches)

    print 'We have a total of',NO_PATCHES,'patches.'
    print 'Errors:',len(all_error_patches)
    print 'Correct:',len(all_correct_patches)    
    
    PATCH_BYTES = 75*75
    P_SIZE = (NO_PATCHES, 4, 75,75) # rather than raveled right now
    
    p_rgba = np.zeros(P_SIZE, dtype=np.float32)
    p_rgba_large = np.zeros(P_SIZE, dtype=np.float32)    
    
#     p_image = np.zeros(P_SIZE, dtype=np.float32)
#     p_prob = np.zeros(P_SIZE, dtype=np.float32)
#     p_binary = np.zeros(P_SIZE, dtype=np.bool)
#     p_merged_array = np.zeros(P_SIZE, dtype=np.bool)
# #     p_dyn_obj = np.zeros((NO_PATCHES, PATCH_BYTES),dtype=np.bool)
# #     p_dyn_bnd = np.zeros((NO_PATCHES, PATCH_BYTES),dtype=np.bool)
#     p_border_overlap = np.zeros(P_SIZE, dtype=np.bool)
#     p_larger_border_overlap = np.zeros(P_SIZE, dtype=np.bool)    
    p_target = np.zeros(NO_PATCHES)


    i = 0
    for p in all_error_patches:
        
        p_rgba[i][0] = (p['image'].astype(np.float32)/255.)
        p_rgba[i][1] = (p['prob'].astype(np.float32)/255.)        
        p_rgba[i][2] = p['merged_array']
        p_rgba[i][3] = p['border_overlap']
        
        p_rgba_large[i][0] = (p['image'].astype(np.float32)/255.)
        p_rgba_large[i][1] = (p['prob'].astype(np.float32)/255.)        
        p_rgba_large[i][2] = p['merged_array']
        p_rgba_large[i][3] = p['larger_border_overlap']        
        
#         p_image[i] = (p['image'].astype(np.float32)/255.)#.ravel()
#         p_prob[i] = (p['prob'].astype(np.float32)/255.)#.ravel()
#         p_binary[i] = p['binary']#.ravel()
#         p_merged_array[i] = p['merged_array']#.ravel()
# #         p_dyn_obj[i] = p['dyn_obj'].ravel()
# #         p_dyn_bnd[i] = p['dyn_bnd'].ravel()
#         p_border_overlap[i] = p['border_overlap']#.ravel()
#         p_larger_border_overlap[i] = p['larger_border_overlap']#.ravel()
        p_target[i] = 1 # <--- important
        i += 1

        
    for p in all_correct_patches:

        p_rgba[i][0] = (p['image'].astype(np.float32)/255.)
        p_rgba[i][1] = (p['prob'].astype(np.float32)/255.)        
        p_rgba[i][2] = p['merged_array']
        p_rgba[i][3] = p['border_overlap']
        
        p_rgba_large[i][0] = (p['image'].astype(np.float32)/255.)
        p_rgba_large[i][1] = (p['prob'].astype(np.float32)/255.)        
        p_rgba_large[i][2] = p['merged_array']
        p_rgba_large[i][3] = p['larger_border_overlap']        
        
#         p_image[i] = (p['image'].astype(np.float32)/255.)#.ravel()
#         p_prob[i] = (p['prob'].astype(np.float32)/255.)#.ravel()
#         p_binary[i] = p['binary']#.ravel()
#         p_merged_array[i] = p['merged_array']#.ravel()
# #         p_dyn_obj[i] = p['dyn_obj'].ravel()
# #         p_dyn_bnd[i] = p['dyn_bnd'].ravel()
#         p_border_overlap[i] = p['border_overlap']#.ravel()
#         p_larger_border_overlap[i] = p['larger_border_overlap']#.ravel()
        p_target[i] = 0 # <--- important
        i+=1
        
    
#     return p_image, p_prob, p_binary, p_merged_array, p_dyn_obj, p_dyn_bnd, p_border_overlap, p_larger_border_overlap, p_target
#     return p_image, p_prob, p_binary, p_merged_array, p_border_overlap, p_larger_border_overlap, p_target
        return p_rgba, p_rgba_large, p_target



def shuffle_in_unison_inplace(a, b, c):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p], c[p]



def run(start_slice, end_slice, filename):
    
    if not os.path.exists(PATCH_PATH):
        os.makedirs(PATCH_PATH)
    
    p = generate_patches(start_slice, end_slice)
    
    shuffled = shuffle_in_unison_inplace(p[0],
                                         p[1],
                                         p[2]
                                        )
    
    print 'saving..'
    np.savez(PATCH_PATH+filename+'.npz', rgba=shuffled[0],
                                         rgba_large=shuffled[1])
    np.savez(PATCH_PATH+filename+'_targets.npz', targets=shuffled[2])
    print 'Done!'
    
#     return correct, split, shuffled


In [8]:
run(40,50,'training')
run(60,62,'test')

working on slice 40
Loading z00000040.png
Generated 1132 split error patches and 1132  correct patches in 351.006586075 seconds.
working on slice 41
Loading z00000041.png
Generated 1220 split error patches and 1220  correct patches in 363.638398886 seconds.
working on slice 42
Loading z00000042.png
Generated 1208 split error patches and 1208  correct patches in 383.692703962 seconds.
working on slice 43
Loading z00000043.png
Generated 1052 split error patches and 1052  correct patches in 383.50474 seconds.
working on slice 44
Loading z00000044.png
Generated 1204 split error patches and 1204  correct patches in 424.27141881 seconds.
working on slice 45
Loading z00000045.png
Generated 1236 split error patches and 1236  correct patches in 418.468515158 seconds.
working on slice 46
Loading z00000046.png
Generated 1108 split error patches and 1108  correct patches in 393.774431944 seconds.
working on slice 47
Loading z00000047.png
Generated 1244 split error patches and 1244  correct patches

In [6]:
shuffled

NameError: name 'shuffled' is not defined

In [9]:
input_image, input_prob, input_mask, input_gold, input_rhoana = mlp.Util.read_section('/home/d/data/cylinder/',50)

error_patches, patches = mlp.Patch.patchify_maxoverlap(input_image, input_prob, input_mask, input_rhoana, input_gold, sample_rate=1)


Loading z00000040.png


In [10]:
error_patches[0]['image'].max()

229

In [12]:
(patches[0]['image'].astype(np.float32)/255.)

array([[ 0.58823532,  0.57647061,  0.56078434, ...,  0.57647061,
         0.58431375,  0.60392159],
       [ 0.59607846,  0.58823532,  0.5529412 , ...,  0.59607846,
         0.61176473,  0.63921571],
       [ 0.57647061,  0.56862748,  0.52156866, ...,  0.6156863 ,
         0.63529414,  0.627451  ],
       ..., 
       [ 0.60784316,  0.6156863 ,  0.63921571, ...,  0.74901962,
         0.74509805,  0.71764708],
       [ 0.67450982,  0.65098041,  0.70588237, ...,  0.75294119,
         0.75686276,  0.74117649],
       [ 0.6901961 ,  0.6901961 ,  0.72941178, ...,  0.76078433,
         0.76078433,  0.74509805]], dtype=float32)

In [14]:
patches[1]['image'].max()

203

In [16]:
for p in error_patches:
    print p['image'].max()

229
229
229
229
221
221
199
199
199
199
199
199
225
225
225
225
194
194
203
203
197
197
187
187
197
197
212
212
226
226
214
214
217
217
212
212
210
210
210
210
209
209
198
198
198
198
202
202
198
198
203
203
214
214
229
229
216
216
216
216
230
230
201
201
231
231
219
219
224
224
220
220
195
195
194
194
194
194
227
227
212
212
217
217
214
214
208
208
208
208
212
212
212
212
214
214
207
207
203
203
214
214
207
207
226
226
216
216
216
216
213
213
204
204
209
209
213
213
203
203
209
209
204
204
194
194
197
197
205
205
205
205
204
204
204
204
203
203
204
204
201
201
200
200
204
204
200
200
198
198
198
198
214
214
222
222
209
209
221
221
221
221
215
215
205
205
204
204
204
204
194
194
194
194
194
194
230
230
216
216
219
219
197
197
197
197
196
196
195
195
203
203
203
203
196
196
218
218
218
218
218
218
224
224
200
200
224
224
203
203
201
201
214
214
195
195
198
198
205
205
213
213
216
216
201
201
218
218
218
218
222
222
218
218
199
199
199
199
201
201
215
215
212
212
184
184
230
230
217
217


In [17]:
len(patches)

1132